# comercio_ext_estatisticas.tb_importacoes
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/IMP_*_delta`

## 📌 Descrição do arquivo

Base de **importações brasileiras**, agregada mensalmente, por produto, país de origem, estado de destino e características logísticas e aduaneiras.  
Não contém identificação de empresas ou CNPJ.

Fonte: MDIC / Comex Stat  
Tipo: Dado público, agregado, não sensível

| Coluna       | Descrição técnica      | Interpretação humana                     |
| ------------ | ---------------------- | ---------------------------------------- |
| `CO_ANO`     | Ano da operação        | Ano em que a importação ocorreu          |
| `CO_MES`     | Mês da operação        | Mês de referência da importação          |
| `CO_NCM`     | Código NCM             | Produto importado                        |
| `CO_UNID`    | Unidade estatística    | Unidade oficial usada para o produto     |
| `CO_PAIS`    | Código do país         | País de origem da mercadoria             |
| `SG_UF_NCM`  | Sigla da UF            | Estado brasileiro de destino             |
| `CO_VIA`     | Código da via          | Meio de transporte utilizado             |
| `CO_URF`     | Código da URF          | Unidade da Receita Federal responsável   |
| `QT_ESTAT`   | Quantidade estatística | Quantidade na unidade estatística do NCM |
| `KG_LIQUIDO` | Peso líquido (kg)      | Peso total importado                     |
| `VL_FOB`     | Valor FOB (US$)        | Valor da mercadoria importada            |
| `VL_FRETE`   | Valor do frete (US$)   | Custo de transporte internacional        |
| `VL_SEGURO`  | Valor do seguro (US$)  | Custo do seguro internacional            |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:

from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable

jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

import re

In [0]:
bronzeBaseDir = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/"
silverPath    = "/mnt/silver/landingbeca2026jan/comercio_ext_estatisticas/tb_importacoes/"
silverTable   = "silver_comercio_ext_estatisticas.tb_importacoes"  # totalmente qualificada p/ DeltaTable.forName
prefix        = "imp_"

status = fs.listStatus(Path(bronzeBaseDir))
dir_names = [s.getPath().getName() for s in status if s.isDirectory()]

pattern = re.compile(rf"{re.escape(prefix)}\d{{4}}")
sourceDirs = [name for name in dir_names if pattern.fullmatch(name)]

if not sourceDirs:
    raise RuntimeError(f"Nenhuma pasta {prefix}YYYY encontrada em {bronzeBaseDir}")

sourcePaths = [bronzeBaseDir.rstrip("/") + "/" + d for d in sourceDirs]

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_ANO",      T.IntegerType(),        nullable=False),
    T.StructField("CO_MES",      T.IntegerType(),        nullable=False),
    T.StructField("CO_NCM",      T.StringType(),         nullable=False),
    T.StructField("CO_UNID",     T.StringType(),         nullable=True ),
    T.StructField("CO_PAIS",     T.StringType(),         nullable=False),
    T.StructField("SG_UF_NCM",   T.StringType(),         nullable=True ),
    T.StructField("CO_VIA",      T.StringType(),         nullable=True ),
    T.StructField("CO_URF",      T.StringType(),         nullable=True ),
    T.StructField("QT_ESTAT",    T.DecimalType(18, 2),   nullable=True ),
    T.StructField("KG_LIQUIDO",  T.DecimalType(18, 3),   nullable=True ),
    T.StructField("VL_FOB",      T.DecimalType(18, 2),   nullable=True ),
    T.StructField("VL_FRETE",    T.DecimalType(18, 2),   nullable=True ),
    T.StructField("VL_SEGURO",   T.DecimalType(18, 2),   nullable=True ),
    T.StructField("TS_REF",      T.TimestampType(),      nullable=False),
    T.StructField("NM_ORIGEM",   T.StringType(),         nullable=False),
])

## Extração
> #### **saprk.read**

In [0]:
dfs = []
for p in sourcePaths:
    print(f"Lendo Delta: {p}")
    dfs.append(spark.read.format("delta").load(p))

if len(dfs) == 1:
    df_bronze_raw = dfs[0]
else:
    df_bronze_raw = dfs[0]
    for d in dfs[1:]:
        df_bronze_raw = df_bronze_raw.unionByName(d, allowMissingColumns=True)

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
df_normalized = (
    df_bronze_raw
    .withColumn("CO_ANO",      F.col("CO_ANO").cast(T.IntegerType()))
    .withColumn("CO_MES",      F.col("CO_MES").cast(T.IntegerType()))
    .withColumn("CO_NCM",      F.upper(F.trim(F.col("CO_NCM").cast(T.StringType()))))
    .withColumn("CO_UNID",     F.col("CO_UNID").cast(T.StringType()))
    .withColumn("CO_PAIS",     F.col("CO_PAIS").cast(T.StringType()))
    .withColumn("SG_UF_NCM",   F.upper(F.trim(F.col("SG_UF_NCM").cast(T.StringType()))))
    .withColumn("CO_VIA",      F.col("CO_VIA").cast(T.StringType()))
    .withColumn("CO_URF",      F.col("CO_URF").cast(T.StringType()))
    .withColumn("QT_ESTAT",    F.col("QT_ESTAT").cast(T.DecimalType(18, 2)))
    .withColumn("KG_LIQUIDO",  F.col("KG_LIQUIDO").cast(T.DecimalType(18, 3)))
    .withColumn("VL_FOB",      F.col("VL_FOB").cast(T.DecimalType(18, 2)))
    .withColumn("VL_FRETE",    F.col("VL_FRETE").cast(T.DecimalType(18, 2)))
    .withColumn("VL_SEGURO",   F.col("VL_SEGURO").cast(T.DecimalType(18, 2)))
)

In [0]:
df_with_defaults = (
    df_normalized
    # negativos -> 0
    .withColumn("QT_ESTAT",   F.when(F.col("QT_ESTAT")   < F.lit(0), F.lit(0)).otherwise(F.col("QT_ESTAT")))
    .withColumn("KG_LIQUIDO", F.when(F.col("KG_LIQUIDO") < F.lit(0), F.lit(0)).otherwise(F.col("KG_LIQUIDO")))
    .withColumn("VL_FOB",     F.when(F.col("VL_FOB")     < F.lit(0), F.lit(0)).otherwise(F.col("VL_FOB")))
    .withColumn("VL_FRETE",   F.when(F.col("VL_FRETE")   < F.lit(0), F.lit(0)).otherwise(F.col("VL_FRETE")))
    .withColumn("VL_SEGURO",  F.when(F.col("VL_SEGURO")  < F.lit(0), F.lit(0)).otherwise(F.col("VL_SEGURO")))
    # UF vazia -> null
    .withColumn("SG_UF_NCM", F.when(F.length(F.col("SG_UF_NCM")) == 0, F.lit(None).cast(T.StringType()))
                              .otherwise(F.col("SG_UF_NCM")))
    # metadados
    .withColumn("TS_REF",    F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/IMP_*_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = (
    df_with_defaults
    .filter(
        F.col("CO_ANO").isNotNull() &
        F.col("CO_MES").isNotNull() &
        F.col("CO_NCM").isNotNull() &
        F.col("CO_PAIS").isNotNull()
    )
    .filter(F.col("CO_MES").between(1, 12))
)

In [0]:
df_dedup = df_valid.dropDuplicates(
    ["CO_ANO","CO_MES","CO_NCM","CO_UNID","CO_PAIS","SG_UF_NCM","CO_VIA","CO_URF"]
)


In [0]:
for f in silverSchema.fieldNames():
    if f not in df_dedup.columns:
        df_dedup = df_dedup.withColumn(f, F.lit(None))

def cast_to(schema, df):
    cols = []
    for field in schema.fields:
        cols.append(F.col(field.name).cast(field.dataType).alias(field.name))
    return df.select(*cols)

df_silver = cast_to(silverSchema, df_dedup)

##Carga
> #### **overwrite**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_estatisticas.tb_importacoes")

merge_condition = """
  t.CO_ANO    = s.CO_ANO   AND
  t.CO_MES    = s.CO_MES   AND
  t.CO_NCM    = s.CO_NCM   AND
  t.CO_UNID   = s.CO_UNID  AND
  t.CO_PAIS   = s.CO_PAIS  AND
  t.SG_UF_NCM = s.SG_UF_NCM AND
  t.CO_VIA    = s.CO_VIA   AND
  t.CO_URF    = s.CO_URF
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "QT_ESTAT":   "s.QT_ESTAT",
        "KG_LIQUIDO": "s.KG_LIQUIDO",
        "VL_FOB":     "s.VL_FOB",
        "VL_FRETE":   "s.VL_FRETE",
        "VL_SEGURO":  "s.VL_SEGURO",
        "TS_REF":     "s.TS_REF",
        "NM_ORIGEM":  "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)

In [0]:
#display(spark.sql("select * from silver_comercio_ext_estatisticas.tb_importacoes"))